In [5]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

# Read the CSV file into a DataFrame
df = pd.read_csv("vehicle_data.csv")

# Extract unique vehicle types
vehicle_types = df['Vehicle_Type(int)'].unique()

# Number of lanes (you can adjust this based on your requirements)
num_lanes = 4

# Processing time matrix showing how much time it would take to process if a vehicle type (i) is passing through a lane (j)
processing_time_matrix = np.array([
        [10, 15, 20, 25],
        [20, 35, 50, 90],
        [35, 70, 80, 90],
        [120, 140, 90, 75]
]).T

#lane switching time
lane_switch_time = 5
#dictionary to convert vehicle type from string to integer
vehicle_type_int = {
        'Motorcycle': 0,
        'Car': 1,
        'Bus': 2,
        'Truck': 3
}
# Genetic Algorithm Parameters
population_size = 20
generations = 50
crossover_rate = 0.4
mutation_rate = 0.1
best_parent_fitness= []

def initialize_population():
    population = []
    for _ in range(population_size):
        lane_assignment = np.random.permutation(len(vehicle_types))
        while any((np.array_equal(lane_assignment, ind) for ind in population)):
            lane_assignment = np.random.permutation(len(vehicle_types))
        population.append(lane_assignment)
    return population

def calculate_fitness(individual):
    # Calculate fitness based on the sum of service times for each lane
    total_service_time = 0
    for index, row in df.iterrows():
        total_service_time += processing_time_matrix[row['Vehicle_Type(int)']][individual[row['Vehicle_Type(int)']]]

    # print('individual :',individual, ', total_service_time :', total_service_time)
    return -total_service_time

def crossover(parent1, parent2):
    mask = np.random.rand(len(vehicle_types)) < 0.5
    child1 = np.where(mask, parent1, parent2)
    child2 = np.where(mask, parent2, parent1)
    return child1, child2

def mutate(individual):
    if np.random.rand() < mutation_rate:
        index_to_mutate = np.random.randint(len(individual))
        individual[index_to_mutate] = np.random.permutation(range(num_lanes))[0]
    return individual

def select_parents(population):
    fitness = [calculate_fitness(individual) for individual in population]
    fitness = np.sort(fitness)
    best_parent_fitness.append(fitness[-1]*-1)
    # print(fitness[0], fitness[-1])
    selected_indices = np.argsort(fitness)[-int(crossover_rate * population_size):]
    # print(selected_indices)
    return [population[i] for i in selected_indices]

def genetic_algorithm():
    population = initialize_population()

    for generation in range(generations):
        selected_population = select_parents(population)

        # Perform crossover
        offspring = []
        for _ in range(population_size // 2):
            parent1, parent2 = random.sample(selected_population, 2)
            child1, child2 = crossover(parent1, parent2)
            offspring.append(mutate(child1))
            offspring.append(mutate(child2))

        # Replace the old population with the new offspring
        population = offspring


    best_individual = max(population, key=calculate_fitness)
    return best_individual

def print_lane_assignment(best_assignment):
    # Print the best lane assignment for each vehicle type
    for vehicle_type, lane in zip(vehicle_types, best_assignment):
        print(f"Vehicle Type: {vehicle_type}, Assigned Lane: {lane}")

def plot_avg_service_time_classwise_static(df):
    """
    Plot a bar chart based on the average service time for each vehicle type from the CSV file.
    """
    # Group by 'Vehicle_Type' and calculate the average service time for each group
    average_service_times = df.groupby('Vehicle_Type')['Service_Time'].mean()

    # Calculate the overall average service time
    overall_average = df['Service_Time'].mean()

    # Plot the bar chart
    plt.bar(average_service_times.index, average_service_times.values, color='skyblue')
    plt.xlabel('Vehicle Type')
    plt.ylabel('Average Service Time')
    plt.title('Classwise Average Service Time(With Static Toll-Gate Allocation)')

    # Annotate each bar with its exact value
    for index, value in enumerate(average_service_times.values):
        plt.text(index, value + 0.1, f'{value:.2f}', ha='center', va='bottom')

    plt.show()

#plot average service time for each vehicle type according to new lane assignment
def plot_avg_service_time_classwise_dynamic(df, best_lane_assignment):
    """
    Plot a bar chart based on the average service time for each vehicle type from the CSV file.
    """
    # Group by 'Vehicle_Type' and calculate the average service time for each group
    average_service_times = df.groupby('Vehicle_Type')['Service_Time'].mean()
    # Calculate the overall average service time
    overall_average = df['Service_Time'].mean()

    # Plot the bar chart
    plt.bar(average_service_times.index, average_service_times.values, color='skyblue')
    plt.xlabel('Vehicle Type')
    plt.ylabel('Average Service Time (Static)')
    plt.title('Classwise Average Service Time(With Static Toll-Gate Allocation)')

    # Annotate each bar with its exact value
    for index, value in enumerate(average_service_times.values):
        plt.text(index, value + 0.1, f'{value:.2f}', ha='center', va='bottom')

    plt.show()

    #plot average service time for each vehicle type according to initial lane assignment
    # Group by 'Vehicle_Type' and calculate the average service time for each group
    #update service time according to new lane assignment
    df['Service_Time'] = df.apply(lambda x: processing_time_matrix[x['Vehicle_Type(int)']][best_lane_assignment[x['Vehicle_Type(int)']]], axis=1)
    average_service_times = df.groupby('Vehicle_Type')['Service_Time'].mean()
    # Calculate the overall average service time
    overall_average = df['Service_Time'].mean()

    # Plot the bar chart
    plt.bar(average_service_times.index, average_service_times.values, color='skyblue')
    plt.xlabel('Vehicle Type')
    plt.ylabel('Average Service Time (Dynamic)')
    plt.title('Classwise Average Service Time(With Dynamic Toll-Gate Allocation)')
    
    for index, value in enumerate(average_service_times.values):
        plt.text(index, value + 0.1, f'{value:.2f}', ha='center', va='bottom')
    
    plt.show()

def evaluate_lane_assignment(lane_assignment):
    # Calculate the total service time for the best lane assignment
    print(lane_assignment)
    
    #assign final lane value according to the best lane assignment
    df['Final_Lane'] = df['Vehicle_Type(int)'].apply(lambda x: lane_assignment[x])
    
    # print(df[['Vehicle_Type', 'Vehicle_Type(int)', 'Initial_Lane', 'Final_Lane']])

def plot_best_fitness(best_parent_fitness):
    plt.plot(best_parent_fitness)
    plt.xlabel('Generations')
    plt.ylabel('Best Fitness')
    plt.title('Best Fitness vs Generations')
    plt.show()

if __name__ == "__main__":
    # plot_avg_service_time_classwise_static(df)
    best_lane_assignment = genetic_algorithm()
    
    print_lane_assignment(best_lane_assignment)
    
    plot_avg_service_time_classwise_dynamic(df, best_lane_assignment)

    plot_best_fitness(best_parent_fitness)

ModuleNotFoundError: No module named 'matplotlib'